In [3]:
import random
import feedback as fb

In [4]:
class Cache( fb.Component ):
    def __init__( self, size, demand ):
        self.t = 0        # internal time counter, needed for last access time
        
        self.size = size  # size limit of cache
        self.cache = {}   # actual cache: cache[key] = last_accessed_time

        self.demand = demand # demand function

    def work( self, u ):
        self.t += 1

        self.size = max( 0, int(u) ) # non-negative integer

        i = self.demand( self.t )    # this is the "requested item"

        if i in self.cache:
            self.cache[i] = self.t   # update last access time
            return 1

        if len(self.cache) >= self.size: # must make room
            m = 1 + len(self.cache) - self.size # number of elements to delete

            tmp = {}
            for k in self.cache.keys():    # key by last_access_time
                tmp[ self.cache[k] ] = k
                
            for t in sorted( tmp.keys() ): # delete the oldest elements
                del self.cache[ tmp[t] ]
                m -= 1
                if m == 0:
                    break

        self.cache[i] = self.t       # insert into cache
        return 0


class SmoothedCache( Cache ):
    def __init__( self, size, demand, avg ):
        Cache.__init__( self, size,  demand );
        self.f = fb.FixedFilter( avg )
        
    def work( self, u ):
        y = Cache.work( self, u )
        return self.f.work(y)


In [9]:

def statictest(demand_width):
    def demand( t ):
        return int( random.gauss( 0, demand_width ) )

    fb.static_test( SmoothedCache, (0, demand, 100), 150, 100, 5, 3000 )
    

def stepresponse():
    def demand( t ): 
        return int( random.gauss( 0, 15 ) )

    def setpoint( t ):
        return 40

    p = SmoothedCache( 0, demand, 100 )

    fb.step_response( setpoint, p )


def closedloop():
    def demand( t ): 
        return int( random.gauss( 0, 15 ) )
    
    def setpoint( t ):
        if t > 5000:
            return 0.5
        return 0.7

    p = SmoothedCache( 0, demand, 100 )
    c = fb.PidController( 100, 250 )

    fb.closed_loop( setpoint, c, p, 10000 )


def closedloop_jumps():
    def demand( t ):
        if t < 3000:
            return int( random.gauss( 0, 15 ) )
        elif t < 5000:
            return int( random.gauss( 0, 35 ) )
        else:
            return int( random.gauss( 100, 15 ) )
    
    def setpoint( t ):
        return 0.7

    p = SmoothedCache( 0, demand, 100 )
    c = fb.PidController( 270, 7.5 ) # Ziegler-Nichols - closedloop1
#   c = fb.PidController( 100, 4.3 ) # Cohen-Coon - 2
#   c = fb.PidController( 80, 2.0 )  # AMIGO - 3
#   c = fb.PidController( 150, 2 )   # 4

    fb.closed_loop( setpoint, c, p, 100 )


In [10]:
       
if __name__ == '__main__':

    fb.DT = 1

    # statictest(35)  # 5, 15, 35  

    # stepresponse()

    # closedloop()
    
    closedloop_jumps()

0 0 0.7 0.7 194.25 194.25 0.0 0.0 
1 1 0.7 0.7 199.5 199.5 0.0 0.0 
2 2 0.7 0.7 204.75 204.75 0.0 0.0 
3 3 0.7 0.7 210.0 210.0 0.0 0.0 
4 4 0.7 0.7 215.25 215.25 0.0 0.0 
5 5 0.7 0.7 220.5 220.5 0.0 0.0 
6 6 0.7 0.7 225.75 225.75 0.0 0.0 
7 7 0.7 0.7 231.0 231.0 0.0 0.0 
8 8 0.7 0.7 236.25 236.25 0.1111111111111111 0.1111111111111111 
9 9 0.7 0.5888888888888888 210.66666666666663 210.66666666666663 0.1 0.1 
10 10 0.7 0.6 218.16666666666666 218.16666666666666 0.18181818181818182 0.18181818181818182 
11 11 0.7 0.5181818181818181 199.9621212121212 199.9621212121212 0.16666666666666666 0.16666666666666666 
12 12 0.7 0.5333333333333333 208.0530303030303 208.0530303030303 0.15384615384615385 0.15384615384615385 
13 13 0.7 0.5461538461538461 215.61072261072258 215.61072261072258 0.14285714285714285 0.14285714285714285 
14 14 0.7 0.5571428571428572 222.756327006327 222.756327006327 0.2 0.2 
15 15 0.7 0.49999999999999994 211.07775557775554 211.07775557775554 0.25 0.25 
16 16 0.7 0.4499999999999

NameError: name 'quit' is not defined